In [108]:
import requests
from bs4 import BeautifulSoup

In [3]:
base_url = 'https://finance.naver.com/item/sise.nhn'
params = {
    'code': '005930'
}
resp = requests.get(base_url, params=params)
resp

<Response [200]>

In [9]:
soup = BeautifulSoup(resp.text)
day_sise = soup.find('iframe', attrs={'name': 'day'})
day_sise

<iframe bottommargin="0" frameborder="0" height="360" marginheight="0" name="day" scrolling="no" src="/item/sise_time.nhn?code=005930&amp;thistime=20200731095610" style="margin-top:20px;clear:both" title="주요 시세" topmargin="0" width="100%"></iframe>

### 올바른 url

In [122]:
base_url = 'https://finance.naver.com/item/sise_day.nhn'
params = {
    'code': '005930'
}
resp = requests.get(base_url, params=params)
soup = BeautifulSoup(resp.text)



In [13]:
# soup.select('table.type2 tr')

In [121]:
tr_tags = soup.select('table.type2 tr')
len(tr_tags)

16

In [16]:
result_list = []
th_tags = tr_tags[0].find_all('th')#TR tag의 0번째 목록

title_list = []
for th in th_tags:
    title_list.append(th.text)
print(title_list)
    
    

['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']


In [22]:
result_list = []
for tr_tag in tr_tags:
    td_tags = tr_tag.find_all('td')
    if len(td_tags)<7:
        continue
        #td목록이 없으면 다음 번째로 넘어감
    
    data_list = []
    for td in td_tags:
        data_list.append(td.text.strip())
        
    result_list.append(
        dict(zip(title_list, data_list))
    )


In [30]:
def get_day_sise(code, page_num):
    base_url = 'https://finance.naver.com/item/sise_day.nhn'
    
    result_list = []
    for page in range(1, page_num+1):
        params = {
            'code': code,
            'page': page
        }

        resp = requests.get(base_url, params=params)
        soup = BeautifulSoup(resp.text)
        
        tr_tags = soup.select('table.type2 tr')
        th_tags = tr_tags[0].find_all('th')

        title_list = []
        for th in th_tags:
            title_list.append(th.text)
        
        
        for tr_tag in tr_tags:
            data_list = []
            td_tags = tr_tag.find_all('td')
            if len(td_tags)<7:
                continue
            for td in td_tags:
                data_list.append(td.text.strip())

            result_list.append(
                dict(zip(title_list, data_list))
            )

    return result_list

In [33]:
sise = get_day_sise('005930',5)

In [34]:
sise

[{'날짜': '2020.07.31',
  '종가': '58,700',
  '전일비': '300',
  '시가': '59,500',
  '고가': '59,600',
  '저가': '58,400',
  '거래량': '9,508,948'},
 {'날짜': '2020.07.30',
  '종가': '59,000',
  '전일비': '0',
  '시가': '59,700',
  '고가': '60,100',
  '저가': '59,000',
  '거래량': '19,285,354'},
 {'날짜': '2020.07.29',
  '종가': '59,000',
  '전일비': '400',
  '시가': '60,300',
  '고가': '60,400',
  '저가': '58,600',
  '거래량': '36,476,611'},
 {'날짜': '2020.07.28',
  '종가': '58,600',
  '전일비': '3,000',
  '시가': '57,000',
  '고가': '58,800',
  '저가': '56,400',
  '거래량': '48,431,566'},
 {'날짜': '2020.07.27',
  '종가': '55,600',
  '전일비': '1,400',
  '시가': '54,300',
  '고가': '55,700',
  '저가': '54,300',
  '거래량': '21,054,421'},
 {'날짜': '2020.07.24',
  '종가': '54,200',
  '전일비': '100',
  '시가': '54,000',
  '고가': '54,400',
  '저가': '53,700',
  '거래량': '10,994,535'},
 {'날짜': '2020.07.23',
  '종가': '54,100',
  '전일비': '600',
  '시가': '54,700',
  '고가': '54,700',
  '저가': '53,800',
  '거래량': '16,214,932'},
 {'날짜': '2020.07.22',
  '종가': '54,700',
  '전일비': '600',
  '시가

### Pandas Excel(csv) IO

In [32]:
import pandas as pd

In [36]:
sise_df = pd.DataFrame(sise)
sise_df

,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.07.31,"58,700",300,"59,500","59,600","58,400","9,508,948"
1,2020.07.30,"59,000",0,"59,700","60,100","59,000","19,285,354"
2,2020.07.29,"59,000",400,"60,300","60,400","58,600","36,476,611"
3,2020.07.28,"58,600","3,000","57,000","58,800","56,400","48,431,566"
4,2020.07.27,"55,600","1,400","54,300","55,700","54,300","21,054,421"
5,2020.07.24,"54,200",100,"54,000","54,400","53,700","10,994,535"
6,2020.07.23,"54,100",600,"54,700","54,700","53,800","16,214,932"
7,2020.07.22,"54,700",600,"55,300","55,500","54,700","12,885,057"
8,2020.07.21,"55,300","1,100","55,200","55,400","54,800","18,297,260"
9,2020.07.20,"54,200",200,"54,800","54,800","54,000","10,507,530"


In [40]:
# csv (Comma Separated View)
sise_df.to_csv("my_sise.csv", encoding='cp949')

In [43]:
temp = pd.read_csv('my_sise.csv', encoding='cp949', index_col=0)
temp

,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.07.31,"58,700",300,"59,500","59,600","58,400","9,508,948"
1,2020.07.30,"59,000",0,"59,700","60,100","59,000","19,285,354"
2,2020.07.29,"59,000",400,"60,300","60,400","58,600","36,476,611"
3,2020.07.28,"58,600","3,000","57,000","58,800","56,400","48,431,566"
4,2020.07.27,"55,600","1,400","54,300","55,700","54,300","21,054,421"
5,2020.07.24,"54,200",100,"54,000","54,400","53,700","10,994,535"
6,2020.07.23,"54,100",600,"54,700","54,700","53,800","16,214,932"
7,2020.07.22,"54,700",600,"55,300","55,500","54,700","12,885,057"
8,2020.07.21,"55,300","1,100","55,200","55,400","54,800","18,297,260"
9,2020.07.20,"54,200",200,"54,800","54,800","54,000","10,507,530"


In [46]:
# Excel
sise_df.to_excel("my_second_sise.xlsx", encoding='cp949')

In [49]:
temp2 = pd.read_excel("my_second_sise.xlsx", encoding='cp949', index_col=0)
temp2

,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.07.31,"58,700",300,"59,500","59,600","58,400","9,508,948"
1,2020.07.30,"59,000",0,"59,700","60,100","59,000","19,285,354"
2,2020.07.29,"59,000",400,"60,300","60,400","58,600","36,476,611"
3,2020.07.28,"58,600","3,000","57,000","58,800","56,400","48,431,566"
4,2020.07.27,"55,600","1,400","54,300","55,700","54,300","21,054,421"
5,2020.07.24,"54,200",100,"54,000","54,400","53,700","10,994,535"
6,2020.07.23,"54,100",600,"54,700","54,700","53,800","16,214,932"
7,2020.07.22,"54,700",600,"55,300","55,500","54,700","12,885,057"
8,2020.07.21,"55,300","1,100","55,200","55,400","54,800","18,297,260"
9,2020.07.20,"54,200",200,"54,800","54,800","54,000","10,507,530"


# Selenium

In [128]:
from selenium import webdriver

In [129]:
driver = webdriver.Chrome()

In [ ]:
driver.maximize_window()

In [ ]:
driver.get('https://www.naver.com')

In [55]:
driver.title

'NAVER'

In [115]:
group_nav = driver.find_element_by_class_name('group_nav')
group_nav

<selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="3fcc5cad-d4f2-4902-bd61-103421287d69")>

In [116]:
nav_items = group_nav.find_elements_by_class_name('nav_item')
nav_items

[<selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="0841ae0a-3991-40e3-8002-c6ead2207d7d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="ffe7fd44-5645-444d-b522-3fed952468d3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="a6d54f33-621b-4c44-8b5d-72d24808b330")>,
 <selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="7c910c45-66cb-45b9-bd2c-8e0a9adb8798")>,
 <selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="f5482b7e-2001-420b-ba0d-379ff1c23e3e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="8b5c7abe-5624-4ae8-9417-ac4aa207a17b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="bfa924f0-7651-495f-b7ea-c2

In [117]:
result_list= [] 
for nav in nav_items:
    anchor_tag = nav.find_element_by_tag_name('a')
    result_list.append({
        'url': anchor_tag.get_attribute('href'),
        'title': anchor_tag.text
    })

In [64]:
result_list

[{'url': 'https://mail.naver.com/', 'title': '메일'},
 {'url': 'https://section.cafe.naver.com/', 'title': '카페'},
 {'url': 'https://section.blog.naver.com/', 'title': '블로그'},
 {'url': 'https://kin.naver.com/', 'title': '지식iN'},
 {'url': 'https://shopping.naver.com/', 'title': '쇼핑'},
 {'url': 'https://order.pay.naver.com/home', 'title': 'Pay'},
 {'url': 'https://tv.naver.com/', 'title': 'TV'},
 {'url': 'https://dict.naver.com/', 'title': '사전'},
 {'url': 'https://news.naver.com/', 'title': '뉴스'},
 {'url': 'https://finance.naver.com/', 'title': '증권'},
 {'url': 'https://land.naver.com/', 'title': '부동산'},
 {'url': 'https://map.naver.com/', 'title': '지도'},
 {'url': 'https://movie.naver.com/', 'title': '영화'},
 {'url': 'https://vibe.naver.com/?from=naver_main', 'title': '뮤직'},
 {'url': 'https://book.naver.com/', 'title': '책'},
 {'url': 'https://comic.naver.com/', 'title': '웹툰'}]

In [65]:
driver.get_window_size()

{'width': 1936, 'height': 1056}

In [66]:
driver.back()

In [67]:
driver.forward()

In [118]:
search_input = driver.find_element_by_id('query')
search_input

<selenium.webdriver.remote.webelement.WebElement (session="348ff4a059d5cd90e9f669bdefe207de", element="13d86ea9-cf3e-4d32-8df4-1eb7d52a6784")>

In [119]:
search_input.send_keys('방탄소년단')

In [70]:
search_btn = driver.find_element_by_id('search_btn')
search_btn.click()

In [71]:
driver.current_url

'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EB%B0%A9%ED%83%84%EC%86%8C%EB%85%84%EB%8B%A8'

## selenium 3페이지뉴스
브라우저를 핸들링하는 것 !리퀴스트를 날려주는 것이 아님!

In [ ]:
driver = webdriver.Chrome()

In [124]:
result_list = []

for i in range(10):
    url = 'https://search.naver.com/search.naver?query=방탄소년단&where=news&start={}'.format(i*10+1)#페이지 넘어가기
    driver.get(url)#**URL가져오기
    
    a_tags = driver.find_elements_by_class_name('_sp_each_title')
    for a in a_tags:
        result_list.append({
            'url': a.get_attribute('href'),
            'title':a.text
        })

In [112]:
result_list[1], result_list[11], result_list[21], result_list[31]

({'url': 'http://sports.khan.co.kr/news/sk_index.html?art_id=202007310956003&sec_id=540101&pt=nv',
  'title': '방탄소년단 뷔랑 여름휴가 어때?'},
 {'url': 'http://news.heraldcorp.com/view.php?ud=20200731000121',
  'title': '방탄소년단 ‘방방콘’·코심, 코로나 시대의 문화예술 활동으로 필리핀 소개'},
 {'url': 'http://star.mt.co.kr/stview.php?no=2020073009453108321',
  'title': '"특히 태형을 위해서라면" 방탄소년단 뷔에 적극 구애한 에어비엔비'},
 {'url': 'http://www.businesspost.co.kr/BP?command=article_view&num=189549',
  'title': "엔터테인먼트3사 주가 다 올라, '방탄소년단 테마주'도 대체로 상승"})

In [111]:
# driver에서 js 실행시키기.
# js를 알아야한다..
driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')

In [131]:
driver.get('https://www.wadiz.kr/web/wreward/main?keyword=&endYn=ALL&order=recommend')
import time
for i in range(100):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(3)

KeyboardInterrupt: 

In [ ]:
screenshot = diver.get_screenshot_as_png("wadiz.png")

In [ ]:
diver.page_source

# Json with dynamic DOM

# json

In [173]:
base_url = "https://crix-api-cdn.upbit.com/v1/crix/candles/months"
params = {
    'code': 'CRIX.UPBIT.KRW-BTC',
    'count':'200',
}
resp = requests.get(base_url, params=params)

In [141]:
import json

In [174]:
month_candle = json.loads(resp.text)
month_candle

[{'code': 'CRIX.UPBIT.KRW-BTC',
  'candleDateTime': '2020-07-01T00:00:00+00:00',
  'candleDateTimeKst': '2020-07-01T09:00:00+09:00',
  'openingPrice': 10903000.0,
  'highPrice': 13368000.0,
  'lowPrice': 10700000.0,
  'tradePrice': 13110000.0,
  'candleAccTradeVolume': 83274.16530314,
  'candleAccTradePrice': 973772272323.6138,
  'timestamp': 1596179406533,
  'firstDayOfPeriod': '2020-07-01'},
 {'code': 'CRIX.UPBIT.KRW-BTC',
  'candleDateTime': '2020-06-01T00:00:00+00:00',
  'candleDateTimeKst': '2020-06-01T09:00:00+09:00',
  'openingPrice': 11432000.0,
  'highPrice': 12230000.0,
  'lowPrice': 10756000.0,
  'tradePrice': 10903000.0,
  'candleAccTradeVolume': 82842.86826319,
  'candleAccTradePrice': 947354527390.6284,
  'timestamp': 1593561601838,
  'firstDayOfPeriod': '2020-06-01'},
 {'code': 'CRIX.UPBIT.KRW-BTC',
  'candleDateTime': '2020-05-01T00:00:00+00:00',
  'candleDateTimeKst': '2020-05-01T09:00:00+09:00',
  'openingPrice': 10477000.0,
  'highPrice': 12163000.0,
  'lowPrice': 10

In [143]:
month_candle[0]

{'code': 'CRIX.UPBIT.KRW-BTC',
 'candleDateTime': '2020-07-01T00:00:00+00:00',
 'candleDateTimeKst': '2020-07-01T09:00:00+09:00',
 'openingPrice': 10903000.0,
 'highPrice': 13368000.0,
 'lowPrice': 10700000.0,
 'tradePrice': 13038000.0,
 'candleAccTradeVolume': 82973.98783208,
 'candleAccTradePrice': 969841584833.2474,
 'timestamp': 1596172888346,
 'firstDayOfPeriod': '2020-07-01'}

In [145]:
month_candle[0]['lowPrice']

10700000.0

In [147]:
#파이썬 객체를 json객체로 바꿈
month_json = json.dumps(month_candle)
month_json

'[{"code": "CRIX.UPBIT.KRW-BTC", "candleDateTime": "2020-07-01T00:00:00+00:00", "candleDateTimeKst": "2020-07-01T09:00:00+09:00", "openingPrice": 10903000.0, "highPrice": 13368000.0, "lowPrice": 10700000.0, "tradePrice": 13038000.0, "candleAccTradeVolume": 82973.98783208, "candleAccTradePrice": 969841584833.2474, "timestamp": 1596172888346, "firstDayOfPeriod": "2020-07-01"}, {"code": "CRIX.UPBIT.KRW-BTC", "candleDateTime": "2020-06-01T00:00:00+00:00", "candleDateTimeKst": "2020-06-01T09:00:00+09:00", "openingPrice": 11432000.0, "highPrice": 12230000.0, "lowPrice": 10756000.0, "tradePrice": 10903000.0, "candleAccTradeVolume": 82842.86826319, "candleAccTradePrice": 947354527390.6284, "timestamp": 1593561601838, "firstDayOfPeriod": "2020-06-01"}, {"code": "CRIX.UPBIT.KRW-BTC", "candleDateTime": "2020-05-01T00:00:00+00:00", "candleDateTimeKst": "2020-05-01T09:00:00+09:00", "openingPrice": 10477000.0, "highPrice": 12163000.0, "lowPrice": 10121000.0, "tradePrice": 11432000.0, "candleAccTrade

In [148]:
month2 = resp.json()
month2

[{'code': 'CRIX.UPBIT.KRW-BTC',
  'candleDateTime': '2020-07-01T00:00:00+00:00',
  'candleDateTimeKst': '2020-07-01T09:00:00+09:00',
  'openingPrice': 10903000.0,
  'highPrice': 13368000.0,
  'lowPrice': 10700000.0,
  'tradePrice': 13038000.0,
  'candleAccTradeVolume': 82973.98783208,
  'candleAccTradePrice': 969841584833.2474,
  'timestamp': 1596172888346,
  'firstDayOfPeriod': '2020-07-01'},
 {'code': 'CRIX.UPBIT.KRW-BTC',
  'candleDateTime': '2020-06-01T00:00:00+00:00',
  'candleDateTimeKst': '2020-06-01T09:00:00+09:00',
  'openingPrice': 11432000.0,
  'highPrice': 12230000.0,
  'lowPrice': 10756000.0,
  'tradePrice': 10903000.0,
  'candleAccTradeVolume': 82842.86826319,
  'candleAccTradePrice': 947354527390.6284,
  'timestamp': 1593561601838,
  'firstDayOfPeriod': '2020-06-01'},
 {'code': 'CRIX.UPBIT.KRW-BTC',
  'candleDateTime': '2020-05-01T00:00:00+00:00',
  'candleDateTimeKst': '2020-05-01T09:00:00+09:00',
  'openingPrice': 10477000.0,
  'highPrice': 12163000.0,
  'lowPrice': 10

## 와디즈


In [184]:
base_url = 'https://www.wadiz.kr/web/wreward/ajaxGetCardList'
params ={
    'startNum': 0,
    'limit': 100,#갯수 제한
    'order': 'recommend',
    'keyword':'',
    'endYn':'ALL'
}
resp = requests.get(base_url, params= params)
resp

<Response [200]>

In [185]:
data = resp.json()

#_data= data['data']
#len(_data)#dic형태
#_data

# 데이터
정형데이터: 구조화된 데이터
비정형데이터 : binary 파일 등 이미지, 음악, 영상-> 틀이 정해져 있지 않음!

<이미지 저장>
웹에서 이미지가 보인다는건 리퀘스트를 날린것

수집하는 방법 -> 
1)파일에 해당 이미지의 URL을 저장
2)실제 이미지를 내 컴퓨터에 저장하고, 저장된 이미지의 경로를 저장
-> 이미지가 파일이 많아지고 크면, 올리는 메모리의 양이 한정되어 있기 때문에. 비효율적
3)실제 이미지 binary를 저장함.




# 이미지 저장

In [157]:
url = 'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F001%2F2020%2F07%2F31%2F11783389.jpg&type=ofullfill80_80_q75_re2'
resp = requests.get(url)
resp

<Response [200]>

In [158]:
resp.content#content는 binary파일, text str

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00P\x00P\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x01\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x07\x05\x08\x06\xff\xc4\x002\x10\x00\x01\x02\x04\x03\x07\x04\x01\x03\x05\x01\x00\x00\x00\x00\x00\x01\x02\x03\x00\x04\x05\x11\x06\x12!\x07\x131AQa\xa1\x14"R\x912\x15q\xf0\x16#\x81\xb1\xc1\xf1\xff\xc4\x00\x17\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\xff\xc4\x00\x1d\x11\x01\x01\x01\x01\x00\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x11\x02\x12!1\x03\x13A\xff\xda\x00\x0c\x03\x01\x00\x02\x11\x03\x11\x00?\x00\xdf\xe0\x

In [159]:
with open('temp.jpg', 'wb') as f:#write binary
    f.write(resp.content)

# headers

In [160]:
resp = requests.get("https://www.naver.com")
resp

<Response [200]>

In [161]:
resp.request.headers

{'User-Agent': 'python-requests/2.24.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [164]:
headers = {
    'User_Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
    'referer':'https://naver.com'
}

In [165]:
resp =  requests.get('https://naver.com', headers= headers)
resp

<Response [200]>

In [166]:
resp.request.headers#해더 변경하기

{'User-Agent': 'python-requests/2.24.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36', 'referer': 'https://naver.com'}

# 실습
데이터 3일치 


# IT과학의 기사 리스트
n일치 3일치
- 기사 리스트: [{
    'id': ~~,
    'title': ~~~~,
     'img': ~~~,
    'content': ~~~~,
    'comments': [
        {
            'author': ~~~,
            'content': ~~~~
        }
    ]
    
}]

In [195]:
import requests

In [201]:
base_url = 'https://news.naver.com/main/list.nhn'
params = {
    'mode': 'LS2D',
    'sid2':731,
    'sid1':105,
    'mid':'shm',
    'date':20200730,
    'page':4
}

resp = requests.get(base_url, params= params)
resp

<Response [200]>

In [203]:
soup = BeautifulSoup(resp.text)
ul_tags = soup.find('ul', class_='type06_headline')
news_li = ul_tags.find_all('li', recursive=False)
print(len(news_li))
news_li

10


[<li>
 <dl>
 <dt class="photo">
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=105&amp;sid2=731&amp;oid=138&amp;aid=0002089447">
 <img alt="[일문일답] LGU+, 비대면 판매채널 공략 “온라인 매출 3배 늘린다”" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/138/2020/07/30/2089447.jpg?type=nf106_72" width="106"/>
 </a>
 </dt>
 <dt>
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=105&amp;sid2=731&amp;oid=138&amp;aid=0002089447">
 								 [일문일답] LGU+, 비대면 판매채널 공략 “온라인 매출 3배 늘린다”
 								</a>
 </dt>
 <dd>
 <span class="lede">[디지털데일리 최민지기자] 코로나19 이후 온라인 구매 경험이 폭발적으로 늘어난 가운데, MZ세대는 실시간 라이브 쇼핑이라는 새 …</span>
 <span class="writing">디지털데일리</span>
 <span class="date is_outdated">1일전</span>
 </dd>
 </dl>
 </li>,
 <li>
 <dl>
 <dt class="photo">
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=105&amp;sid2=731&amp;oid=003&am

In [221]:
result_list = []

for news in news_li:
    title_tag = news.find('a', class_= "")
    title = title_tag.text
    url = title_tag.get('href')
    
    desc = news.find_all('dd')
    desc = desc[1].text
    result_list.append({
        'title': title,
        'url': url,
        'desc': desc
    })
print(len(result_list))
result_list

IndexError: list index out of range

In [211]:
result_list=[]

titles = soup.select('ul.type06_headline>li>dl>dt>a')
title_tag = news.find('a', class_ = "")
url = title_tag.get('href')
    
    #desc = news.find_all('dd')
    #  desc = desc[1].text
    result_list.append({
        'title': titles,
        'url': url
    })
print(len(result_list))
result_list

IndentationError: unexpected indent (<ipython-input-211-26d7bd6fa721>, line 9)